In [1]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
import openai
import time
import logging
import json
from datetime import datetime

In [2]:
load_dotenv()
MYSQL_URI = os.getenv("MYSQL_URI")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
engine = create_engine(MYSQL_URI)

In [3]:
client = openai.OpenAI()
model  = 'gpt-4o-mini'
client_id = 100000

In [4]:
def get_accounts(client_id):
    accounts = pd.read_sql(con=engine, sql=f'SELECT * FROM Accounts WHERE ClientID={client_id}')
    accounts.drop(columns=['ClientID', 'CreatedBy'])
    return accounts

In [5]:
def get_leads(client_id):
    leads = pd.read_sql(con=engine, sql=f'SELECT * FROM Leads WHERE ClientID={client_id}')
    accounts = pd.read_sql(con=engine, sql=f'SELECT AccountID, CompanyName FROM Accounts WHERE ClientID={client_id}')
    leads = pd.merge(leads, accounts, on='AccountID', how='left')
    leads = leads.drop(columns=['ClientID', 'CreatedBy'])
    return leads

In [6]:
leads = get_leads(client_id)
accounts = get_accounts(client_id)

In [7]:
# filepath = f'static/files/{client_id}'

# os.makedirs(filepath, exist_ok=True)

# leads.to_json(path_or_buf=f'static/files/{client_id}/leads.json', orient='records')
# accounts.to_json(path_or_buf=f'static/files/{client_id}/accounts.json', orient='records')

In [8]:
assistant_id = 'asst_MkWTjEDWaQ8N4RQ7TRj9RyEl'

assistant = client.beta.assistants.retrieve(assistant_id=assistant_id)

In [9]:
# vector_store = client.beta.vector_stores.create(name="Tables")

In [10]:
# file_stream = open(f'{filepath}/leads.json', 'rb')
# file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
#     vector_store_id=vector_store.id,
#     files=[file_stream]
# )

In [11]:
# file_batch.status

In [12]:
# for file in list:
#     client.beta.vector_stores.files.delete(
#         vector_store_id=vector_store_id,
#         file_id=file)

In [13]:
instructions = 'You are a CRM assistant. Your tasks are: \
            1.	Analyze leads based on their data to provide insights into their potential value. Provide ordered lists of strong leads and reasoning for why these could be potential clients.\
                Provide as much information for each lead as possible. You should not reveal that you are a chatbot in any way. Include at least: lead position, company, and email (if email)'
            # 2.	Review interactions with leads to generate personalized sales scripts tailored to their needs and preferences.'

In [14]:
# assistant = client.beta.assistants.update(
#     assistant_id=assistant_id,
#     model=model,
#     instructions=instructions,
#     tools=[{'type': 'file_search'}],
#     # tools=[
#     #     {
#     #     'type': 'function',
#     #     'function': {
#     #         'name': 'get_leads',
#     #         'description': 'Retrieve list (DataFrame) of leads from database.',
#     #         'parameters': {
#     #             'type': 'object',
#     #             'properties': {
#     #                 'client_id':{
#     #                     'type': 'integer',
#     #                     'description': 'ClientID of the assistant\'s user. Use to filter database (SQL WHERE clause).',
#     #                 }
#     #             },
#     #             'required': ['client_id'],
#     #         },
#     #        }       
#     #     },
#     # ],
#     tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
# )

In [15]:

# assistant = client.beta.assistants.create(
#     name='CRM Assistant',
#     instructions='You are a CRM assistant. Your tasks are: \
#         1.	Analyze leads based on their data to provide insights into their potential value. Provide ordered lists of strong leads and reasoning for why these could be potential clients.\
#         2.	Review interactions with leads to generate personalized sales scripts tailored to their needs and preferences.',
#     model=model,
#     tools=[
#         # {
#         # 'type': 'function',
#         # 'function': {
#         #     'name': 'get_accounts',
#         #     'description': 'Retrieve list (DataFrame) of accounts from database.',
#         #     'parameters': {
#         #         'type': 'object',
#         #         'properties': {
#         #             'client_id':{
#         #                 'type': 'integer',
#         #                 'description': 'ClientID of the assistant\'s user. Use to filter database (SQL WHERE clause).',
#         #             }
#         #         },
#         #         'required': ['client_id'],
#         #     },
#         #    }       
#         # },
#         {
#         'type': 'function',
#         'function': {
#             'name': 'get_leads',
#             'description': 'Retrieve list (DataFrame) of leads from database. Each lead is associated with a certain account.',
#             'parameters': {
#                 'type': 'object',
#                 'properties': {
#                     'client_id':{
#                         'type': 'integer',
#                         'description': 'ClientID of the assistant\'s user. Use to filter database (SQL WHERE clause).',
#                     }
#                 },
#                 'required': ['client_id'],
#             },
#            }       
#         },
#     ]
#     # file_ids=
# )

In [16]:
thread = client.beta.threads.create()

In [17]:
# message = client.beta.threads.messages.create(
#     thread_id=thread.id,
#     role='user',
#     content='How many accounts are there?'
# )


In [18]:
# # run: the users client id is 100000
# run = client.beta.threads.runs.create_and_poll(
#     thread_id=thread.id,
#     assistant_id=assistant.id,
#     instructions='Search the database and answer the users questions with the relevant information. Users ClientID is 100000'
# )

In [19]:

def call_function(required_actions, run):
    tool_outputs = []
    for action in required_actions['tool_calls']:
        function_name = action['function']['name']
        arguments = json.loads(action['function']['arguments'])
        
        # if function_name == 'get_accounts':
        #     output = get_accounts(client_id=arguments['client_id'])
        #     output = output.to_csv(index=True)
        #     print(output)
            
        if function_name == 'get_leads':
            output = get_leads(client_id=arguments['client_id'])
            # output.to_csv(f'static/files/{client_id}.txt', index=False)
            output = output.to_csv(index=False)
            print(output)
            
        tool_outputs.append({'tool_call_id': action['id'],
                             'output': output})
        
    print('Submitting output back to the assistant.')  
    client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread.id,
        run_id=run,
        tool_outputs=tool_outputs
    )

In [20]:
def add_message(content):
        if thread:
            client.beta.threads.messages.create(
                thread_id=thread.id,
                role='user',
                content=content
            )

In [21]:
def run_assistant():
    run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions='Provide an ordered list of relevant potential clients and reason why. Provide as much information for each lead as possible.\
        You should not reveal that you are a chatbot in any way.')
    return run

In [22]:
def process_messages():
        if thread:
            messages = client.beta.threads.messages.list(
                thread_id=thread.id
            )

            last_message = messages.data[0]
            role = last_message.role
            response = last_message.content[0].text.value
            
            return f'{role.capitalize()}:\n{response}'

In [23]:

def generate(prompt):
    if thread:
        add_message(prompt)
        run = run_assistant()
        if run:
            while True:
                time.sleep(1)
                run_status = client.beta.threads.runs.retrieve(
                    thread_id=thread.id,
                    run_id=run.id
                )
                print(run_status.status)
                if run_status.status == 'completed':
                    elapsed_time = run_status.completed_at - run_status.created_at
                    formatted_elapsed_time = time.strftime(
                        "%H:%M:%S", time.gmtime(elapsed_time)
                    )
                    response = process_messages()
                    print(response)
                    return f"Run completed in {formatted_elapsed_time}\n{response}"
                elif run_status.status == 'requires_action':
                    print('Function calling in progress.')
                    call_function(
                        required_actions = run_status.required_action.submit_tool_outputs.model_dump(),
                        run=run.id 
                    )
                elif run_status.status == 'failed':
                    break
    return 0

In [24]:
x = generate('are there any leads from tumi?')

in_progress
in_progress
in_progress
in_progress
completed
Assistant:
Here are potential leads from Tumi:

1. **Bob Weingartner**
   - **Position**: Vice President Information Technology
   - **Email**: bob.weingartner@samsonite.com
   - **Company**: Tumi
   - **Note**: As the Vice President of IT, Bob is a decision-maker in technology investment and infrastructure, making him a key contact for any technology services or products that Tumi may require【4:1†source】.

2. **Kchitij Kumar**
   - **Position**: Head of Technology & Data Analytics
   - **Email**: kchitij.kumar@tumi.com
   - **Company**: Tumi
   - **Note**: Kchitij oversees technology and data analytics, indicating he would be crucial for discussions around data management solutions or technology enhancements within Tumi【4:1†source】.

Both of these contacts within Tumi are likely to be important for any proposals related to IT solutions, technology analytics, and infrastructure improvements.


In [25]:
print(x)

Run completed in 00:00:06
Assistant:
Here are potential leads from Tumi:

1. **Bob Weingartner**
   - **Position**: Vice President Information Technology
   - **Email**: bob.weingartner@samsonite.com
   - **Company**: Tumi
   - **Note**: As the Vice President of IT, Bob is a decision-maker in technology investment and infrastructure, making him a key contact for any technology services or products that Tumi may require【4:1†source】.

2. **Kchitij Kumar**
   - **Position**: Head of Technology & Data Analytics
   - **Email**: kchitij.kumar@tumi.com
   - **Company**: Tumi
   - **Note**: Kchitij oversees technology and data analytics, indicating he would be crucial for discussions around data management solutions or technology enhancements within Tumi【4:1†source】.

Both of these contacts within Tumi are likely to be important for any proposals related to IT solutions, technology analytics, and infrastructure improvements.
